[![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/worldcoin/open-iris/blob/dev/colab/MatchingEntities.ipynb)

# IRIS: _Matching entities_ tutorial

This Jupyter Notebook will walk you through the basics of how to use matchers available in the `iris` package. From it you will learn how to:
- Use the `HammingDistanceMatcher` matcher to compute distance between two eyes.

## 0. Installing `iris` package in the `SERVER` environment.

Install `iris` package

In [ ]:
!pip install open-iris

Verify installation by printing `iris` version

In [ ]:
import iris

print(iris.__version__)

Download sample IR images for experimenting.

In [ ]:
!wget https://wld-ml-ai-data-public.s3.amazonaws.com/public-iris-images/example_orb_image_1.png -O ./subject1_first_image.png
!wget https://wld-ml-ai-data-public.s3.amazonaws.com/public-iris-images/example_orb_image_2.png -O ./subject1_second_image.png
!wget https://wld-ml-ai-data-public.s3.amazonaws.com/public-iris-images/example_orb_image_3.png -O ./subject2_image.png

## 1. Use the `HammingDistanceMatcher` matcher to compute distance between two eyes.

Load all IR images with `opencv-python` package. 

In [ ]:
import cv2

subject1_first_image = cv2.imread("./subject1_first_image.png", cv2.IMREAD_GRAYSCALE)
subject1_second_image = cv2.imread("./subject1_second_image.png", cv2.IMREAD_GRAYSCALE)
subject2_image = cv2.imread("./subject2_image.png", cv2.IMREAD_GRAYSCALE)

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(subject1_first_image, cmap='gray')
plt.title('Subject 1, image 1')
plt.show()

plt.imshow(subject1_second_image, cmap='gray')
plt.title('Subject 1, image 2')
plt.show()

plt.imshow(subject2_image, cmap='gray')
plt.title('Subject 2, image 1')
plt.show()

Create `IRISPipeline` object and compute `IrisTemplate`s for all images.

In [ ]:
import iris

iris_pipeline = iris.IRISPipeline()

output_1 = iris_pipeline(subject1_first_image, eye_side="left")
subject1_first_code = output_1['iris_template']

output_2 = iris_pipeline(subject1_second_image, eye_side="left")
subject1_second_code = output_2['iris_template']

output_3 = iris_pipeline(subject2_image, eye_side="left")
subject2_code = output_3['iris_template']

Create a `HammingDistanceMatcher` matcher object.
```python
def __init__(
        self,
        rotation_shift: int = 15,
        nm_dist: Optional[confloat(ge=0, le=1, strict=True)] = None,
        weights: Optional[List[np.ndarray]] = None,
    ) -> None:
        """Assign parameters.

        Args:
            rotation_shift (int): rotations allowed in matching, converted to shifts in columns. Defaults to 15.
            nm_dist (Optional[confloat(ge=0, le = 1, strict=True)]): nonmatch distance used for normalized HD. Optional paremeter for normalized HD. Defaults to None.
            weights (Optional[List[np.ndarray]]): list of weights table. Optional paremeter for weighted HD. Defaults to None.
        """
```

In [ ]:
matcher = iris.HammingDistanceMatcher()

Call `run` method and provide two `IrisTemplate`s to compute distances.
```python
def run(self, template_probe: IrisTemplate, template_gallery: IrisTemplate) -> float:
```

In [ ]:
same_subjects_distance = matcher.run(subject1_first_code, subject1_second_code)

print(f"Computed distance between eyes of the `subject_1` is {same_subjects_distance:.4f}")

different_subjects_distance = matcher.run(subject1_first_code, subject2_code)

print(f"Computed distance between eyes of the `subject_1` and `subject_2` is {different_subjects_distance:.4f}")

The recommended threshold is between 0.34 and 0.39 depending on the sensibitity of your use case, 0.37 is a good choice if you do not favor False Matches or False Non-Matches.

In this example, the two iris images from the same subject are clearly matched, and those form different subjects clearly non-matched.

### Thank you for making it to the end of this tutorial!